In [29]:
from dotenv import load_dotenv
import requests
import os
from bs4 import BeautifulSoup
from bs2json import bs2json
import pandas as pd 

In [2]:
data = requests.get('https://fbref.com/en/players/dea698d9/all_comps/Cristiano-Ronaldo-Stats---All-Competitions')
player = BeautifulSoup(data.text, 'html.parser')

In [3]:
# Key = table, V = columns I need to take from the table 
tables_cols = {1: ["shots_on_target_pct", "goals", "shots_on_target", "shots_total", "average_shot_distance", "shots_free_kicks", "pens_made", "pens_att"],
               2: ["passes_pct", "passes_completed", "passes_total_distance", "passes_progressive_distance", "passes_pct_short", "passes_pct_medium", "passes_pct_long" ,"assists", "assisted_shots", "passes_into_final_third", "passes_into_penalty_area", "crosses_into_penalty_area", "progressive_passes"],
               5: ["tackles", "tackles_won", "dribble_tackles_pct", "pressure_regain_pct", "blocks", "interceptions", "clearances", "errors"],
               6: ["dribbles_completed_pct", "players_dribbled_past", "carries", "carry_distance", "carry_progressive_distance", "miscontrols", "dispossessed"],
               7: ["points_per_match", "plus_minus", "plus_minus_wowy"],
               8: ["fouled", "pens_conceded", "own_goals", "ball_recoveries", "aerials_won_pct"]}

In [4]:
# Dataframes for appending the values
shooting_df = pd.DataFrame(columns=["shots_on_target_pct", "goals", "shots_on_target", "shots_total", "average_shot_distance", "shots_free_kicks", "pens_made", "pens_att"])

passing_df = pd.DataFrame(columns=["passes_pct", "passes_completed", "passes_total_distance", "passes_progressive_distance", "passes_pct_short", "passes_pct_medium", "passes_pct_long" ,"assists", "assisted_shots", "passes_into_final_third", "passes_into_penalty_area", "crosses_into_penalty_area", "progressive_passes"])

defending_df = pd.DataFrame(columns=["tackles", "tackles_won", "dribble_tackles_pct", "pressure_regain_pct", "blocks", "interceptions", "clearances", "errors"])

dribbling_df = pd.DataFrame(columns=["dribbles_completed_pct", "players_dribbled_past", "carries", "carry_distance", "carry_progressive_distance", "miscontrols", "dispossessed"])

others_df = pd.DataFrame(columns=["points_per_match", "plus_minus", "plus_minus_wowy"])

miscel_df = pd.DataFrame(columns=["fouled", "pens_conceded", "own_goals", "ball_recoveries", "aerials_won_pct"])

In [5]:
cols_scraped = {1: [], 2: [], 5: [], 6: [], 7: [], 8: []}

In [6]:
# All tables
tables = player.find_all('div', {'class': 'section_wrapper'})
# Convert the table to a json
converter = bs2json()

In [7]:
for t in tables_cols:

    # Select the table from the json
    table_s = converter.convert(tables[t])
    # Table as html again
    table = BeautifulSoup(table_s['div']['text'], 'html.parser')
    # Select the table in mode 'Club Collapsed'
    t_collapsed = table.find_all('div', {'class': 'table_wrapper'})[1]
    # Empty list for placing the values to append to the dataframe
    values = []

    for col in tables_cols[t]:
        # Select the season (-3 means 3 seasons ago -> 18/19) MAYBE THERE'S A BETTER WAY OF CHOOSING THIS
        values.append(float(t_collapsed.find('tbody').find_all('tr')[-3].find('td', {'data-stat': col}).text))

    # Append list to cols_scrapped as a row
    cols_scraped[t].append(values)

In [44]:
data = requests.get('https://fbref.com/en/players/7ba6d84e/all_comps/David-de-Gea-Stats---All-Competitions')
player = BeautifulSoup(data.text, 'html.parser')

In [49]:
# All tables
tables = player.find_all('div', {'class': 'section_wrapper'})
# Convert the table to a json
converter = bs2json()

In [50]:
# Select the table from the json
table_s = converter.convert(tables[1])
# Table as html again
table = BeautifulSoup(table_s['div']['text'], 'html.parser')
# Select the table in mode 'Club Collapsed'
t_collapsed = table.find_all('div', {'class': 'table_wrapper'})[1]

In [53]:
t_collapsed.find('tbody').find_all('tr')[-3]

<tr id="stats" style="line-height: 1.3em"><th class="left" data-stat="season" scope="row">2018-2019</th><td class="center" data-stat="age">27</td><td class="left" data-stat="squad"><a href="/en/squads/19538871/2018-2019/Manchester-United-Stats">Manchester Utd</a></td><td class="left" data-stat="comp_level">3 Leagues</td><td class="right" data-stat="minutes_90s_gk">47.0</td><td class="right group_start" data-stat="goals_against_gk">63</td><td class="right" data-stat="pens_allowed">6</td><td class="right" data-stat="free_kick_goals_against_gk">2</td><td class="right" data-stat="corner_kick_goals_against_gk">8</td><td class="right" data-stat="own_goals_against_gk">4</td><td class="right group_start" data-stat="psxg_gk">60.6</td><td class="right" data-stat="psnpxg_per_shot_on_target_against">0.28</td><td class="right plus" data-stat="psxg_net_gk">+1.6</td><td class="right plus" data-stat="psxg_net_per90_gk">+0.03</td><td class="right group_start" data-stat="passes_completed_launched_gk">37